<a href="https://colab.research.google.com/github/Dimildizio/chat-bot-yakutia/blob/main/bot/vk_yak_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install vk_api langchain openai chromadb tiktoken

In [4]:
import vk_api
from vk_api.longpoll import VkLongPoll, VkEventType
from vk_api.utils import get_random_id
from google.colab import userdata
import click
import os
import openai
import chromadb
import tiktoken
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

In [5]:
token = userdata.get('vk_public')
os.environ["OPENAI_API_KEY"] = userdata.get('gpt_api')
openai.api_key = userdata.get('gpt_api')
authorize = vk_api.VkApi(token=token)
longpoll = VkLongPoll(authorize)

In [6]:
def main():
    vk_session = vk_api.VkApi(token=token)
    longpoll = VkLongPoll(vk_session)

    for event in longpoll.listen():
        if event.type == VkEventType.MESSAGE_NEW and event.to_me:
            handle_message(event)

In [7]:
def get_chatgpt_response(message):
    loader = TextLoader('data.txt')
    index = VectorstoreIndexCreator().from_loaders([loader])

    chain = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model='gpt-3.5-turbo'),
        retriever=index.vectorstore.as_retriever(
            search_kwargs={'k': 1}))

    chat_log = []
    result = chain({'question': message, 'chat_history': chat_log})

    chat_log.append((message, result['answer']))
    return result['answer']


In [8]:
def send_message(vk, user_id, text):
    vk.method('messages.send', {'user_id': user_id, 'message': text, 'random_id': get_random_id()})

In [9]:
def handle_message(event):
    vk = vk_api.VkApi(token=token)
    print('User input: ',event.text)
    response = get_chatgpt_response(event.text)
    print('output: ', response)
    send_message(vk, event.user_id, response)

In [ ]:
try:
  main()
except KeyboardInterrupt:
    print("Bot stopped.")